In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from functools import partial
import subprocess
import time
import re

def run_java_anonymization(input_csv, output_csv, k_value, l_value=2, weights=None):
    """
    Führt das Java-Programm zur Anonymisierung mit den angegebenen Parametern aus.
    
    Args:
        input_csv (str): Pfad zur Eingabe-CSV-Datei.
        output_csv (str): Pfad zur Ausgabe-CSV-Datei.
        k_value (str): Wert für k-Anonymität.
        l_value (int, optional): Wert für l-Diversität (Standard: 2).
        weights (dict, optional): Gewichtungen für die Attribute, falls vorhanden.
    """
    start_time = time.time()

    # Pfade und Parameter
    java_file = "C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/src/main/java/ARXAnonymization_w.java"
    class_file = "C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/src/main/java/ARXAnonymization_w.class"
    libs_dir = "C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/lib/*"

    # Gewichtungen als String formatieren, wenn vorhanden
    if weights is not None:
        weights_str = ','.join([str(weights[key]) for key in sorted(weights.keys())])
    else:
        weights_str = ""

    run_command = [
        "java",
        "-cp", f"{libs_dir};C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/src/main/java",
        "ARXAnonymization_w",
        input_csv,
        output_csv,
        str(k_value),
        str(l_value),
        weights_str
    ]

    print(f"Running Java code with k={k_value}...")
    process = subprocess.Popen(run_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    while True:
        output = process.stdout.readline()
        if output == "" and process.poll() is not None:
            break
        if output:
            print(output.strip())

    stderr_output = process.stderr.read()
    if stderr_output:
        print(f"Error:\n{stderr_output}")

    exit_code = process.poll()
    if exit_code == 0:
        print("Execution successful!")
    else:
        print(f"Execution failed with exit code {exit_code}")

    # Überprüfen, ob die Datei erstellt wurde
    if not os.path.exists(output_csv):
        raise FileNotFoundError(f"Anonymisierungsdatei {output_csv} für k={k_value} wurde nicht erstellt.")

    end_time = time.time()
    elapsed_time = end_time - start_time
    hours_total, rem_total = divmod(elapsed_time, 3600)
    minutes_total, seconds_total = divmod(rem_total, 60)
    print(f"Gesamte Ausführung hat {int(hours_total)} Stunden, {int(minutes_total)} Minuten, und {seconds_total:.2f} Sekunden gedauert.")
    
    return exit_code

def anonymize_data(df, weights, trial_number, temp_input_csv="C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_input.csv", temp_output_csv_template="C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_{}_k{}.csv"):
    df.to_csv(temp_input_csv, index=False)
    k_values = [10, 20]

    for k_value in k_values:
        temp_output_csv = temp_output_csv_template.format(trial_number, k_value)
    
        print(f"Running Java anonymization for trial {trial_number} with k={k_value}...")
        java_exit_code = run_java_anonymization(temp_input_csv, temp_output_csv, k_value, weights=weights)

        if java_exit_code != 0:
            print(f"Anonymization script failed with exit code {java_exit_code} for k={k_value}.")
            continue

        for i in range(10):
            if os.path.exists(temp_output_csv):
                break
            time.sleep(1)
        else:
            print(f"{temp_output_csv} wurde nicht gefunden. Erstelle eine leere Datei.")
            pd.DataFrame().to_csv(temp_output_csv, index=False)
            raise FileNotFoundError(f"{temp_output_csv} wurde nicht gefunden.")
    
        anonymized_df = pd.read_csv(temp_output_csv)
        print(f"Anonymization completed for k={k_value}. Output saved to {temp_output_csv}.")

        if anonymized_df.empty or 'cardio' not in anonymized_df.columns or anonymized_df['cardio'].isnull().all():
            print(f"Anonymisierte Daten für k={k_value} enthalten keine nutzbaren Informationen. Überspringe Klassifikation.")
            return pd.DataFrame()  # Rückgabe eines leeren DataFrame, um die Klassifikation zu überspringen

    print("All anonymization processes completed.")
    return anonymized_df

# Funktion zur Bereinigung der 'weight'-Werte
def convert_weight_value(x):
    # Überprüfen, ob der Wert ein Bereich ist, z.B. "[120. 160["
    match = re.match(r"\[(\d+\.?\d*)\s+(\d+\.?\d*)\[" , str(x))
    if match:
        # Den Mittelwert des Bereichs berechnen
        lower_bound = float(match.group(1))
        upper_bound = float(match.group(2))
        return (lower_bound + upper_bound) / 2
    elif x == '*':
        return None  # Platzhalter für fehlende Werte
    else:
        # Ansonsten versuchen, den Wert als Float zu interpretieren
        try:
            return float(x)
        except ValueError:
            return None

def process_value(value):
    if isinstance(value, str):
        value = value.strip()
        if value.startswith('[') and value.endswith('['):
            try:
                start, end = map(float, value[1:-1].split(','))
                return (start + end) / 2
            except ValueError:
                return np.nan
        elif value.startswith('{') and value.endswith('}'):
            try:
                values = list(map(float, value[1:-1].split(',')))
                return np.mean(values)
            except ValueError:
                return np.nan
        else:
            try:
                return float(value)
            except ValueError:
                return np.nan
    return value


def evaluate_model(df, weights, trial_number):
    df_anonymized = anonymize_data(df, weights, trial_number)
    # Wenn der anonymisierte DataFrame leer ist, Klassifikation überspringen
    if df_anonymized.empty:
        print(f"Anonymisierung für Trial {trial_number} fehlgeschlagen oder keine nutzbaren Daten vorhanden.")
        return 0

    df_anonymized = df_anonymized.drop(columns=["id"], errors='ignore')
    df_anonymized = df_anonymized[~df_anonymized.isin(['*']).any(axis=1)]
    df_anonymized = df_anonymized.dropna()
    
    X = df_anonymized.drop(columns=["cardio"], errors='ignore')
    y = df_anonymized["cardio"]

    for column in X.columns:
        X[column] = X[column].apply(process_value)

    if X.empty or y.empty:
        print(f"Fehler: Keine nutzbaren Daten nach Anonymisierung in Trial {trial_number}.")
        return 0

    clf = RandomForestClassifier()
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    return scores.mean()

def objective(trial, df):
    weights = {
        "age": trial.suggest_float("age_weight", 0.0, 1.0),
        "gender": trial.suggest_float("gender_weight", 0.0, 1.0),
        "height": trial.suggest_float("height_weight", 0.0, 1.0),
        "weight": trial.suggest_float("weight_weight", 0.0, 1.0),
        "ap_hi": trial.suggest_float("ap_hi_weight", 0.0, 1.0),
        "ap_lo": trial.suggest_float("ap_lo_weight", 0.0, 1.0),
        "cholesterol": trial.suggest_float("cholesterol_weight", 0.0, 1.0),
        "gluc": trial.suggest_float("gluc_weight", 0.0, 1.0),
        "smoke": trial.suggest_float("smoke_weight", 0.0, 1.0),
        "alco": trial.suggest_float("alco_weight", 0.0, 1.0),
        "active": trial.suggest_float("active_weight", 0.0, 1.0),
    }

    if any(np.isnan(list(weights.values()))):
        print("Es gibt ungültige Gewichtungen in den Attributen.")
        return 0

    score = evaluate_model(df, weights, trial.number)
    return score

def optimize_weights_for_datasets(output_file="C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/optimized_results.csv"):
    datasets = ["C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/cvd_k0_l0.csv",
                "C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/cvd_k10_l2.csv",
                "C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/cvd_k100_l2.csv",
                "C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/cvd_k1000_l2.csv"
               ]
    results = []

    for dataset in datasets:
        print(f"Optimizing for {dataset}")
        df = pd.read_csv(dataset, delimiter=';')
        df['weight'] = df['weight'].str.replace(',', '.').apply(convert_weight_value)

        study = optuna.create_study(direction="maximize")
        study.optimize(partial(objective, df=df), n_trials=50)

        print(f"Optimized weights for {dataset}: {study.best_params}")
        
        # Ergebnis speichern (beste Gewichtungen und Score)
        result = {
            "dataset": dataset,
            "best_weights": study.best_params,
            "best_score": study.best_value
        }
        results.append(result)

    # Ergebnisse in eine CSV-Datei speichern
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"Optimization results saved to {output_file}.")

output_file = "C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/optimized_results.csv"
optimize_weights_for_datasets(output_file)


[I 2024-09-17 19:44:26,729] A new study created in memory with name: no-name-c5e87408-dce6-4dc0-b156-50fb74bc6887


Optimizing for C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/cvd_k0_l0.csv
Running Java anonymization for trial 0 with k=10...
Running Java code with k=10...
Anonymization successful for k = 10, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 32.93 Sekunden gedauert.
Anonymization completed for k=10. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_0_k10.csv.
Running Java anonymization for trial 0 with k=20...
Running Java code with k=20...
Anonymization successful for k = 20, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 32.76 Sekunden gedauert.
Anonymization completed for k=20. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_0_k20.csv.
All anonymization processes completed.


[I 2024-09-17 19:45:42,615] Trial 0 finished with value: 0.7198266976990588 and parameters: {'age_weight': 0.849267429030091, 'gender_weight': 0.3371963365626721, 'height_weight': 0.9055086736048579, 'weight_weight': 0.7810336703300543, 'ap_hi_weight': 0.12340100949463606, 'ap_lo_weight': 0.38899307097095304, 'cholesterol_weight': 0.009341277459541386, 'gluc_weight': 0.8736676236714979, 'smoke_weight': 0.4174092098645704, 'alco_weight': 0.1716774960334141, 'active_weight': 0.3268498074996268}. Best is trial 0 with value: 0.7198266976990588.


Running Java anonymization for trial 1 with k=10...
Running Java code with k=10...
Anonymization successful for k = 10, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 32.70 Sekunden gedauert.
Anonymization completed for k=10. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_1_k10.csv.
Running Java anonymization for trial 1 with k=20...
Running Java code with k=20...
Anonymization successful for k = 20, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 32.66 Sekunden gedauert.
Anonymization completed for k=20. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_1_k20.csv.
All anonymization processes completed.


[I 2024-09-17 19:46:58,594] Trial 1 finished with value: 0.7108969664222344 and parameters: {'age_weight': 0.6926961740009183, 'gender_weight': 0.1521592024905315, 'height_weight': 0.1358133274792731, 'weight_weight': 0.5983663058757148, 'ap_hi_weight': 0.9345543581242608, 'ap_lo_weight': 0.9254424138436175, 'cholesterol_weight': 0.3764406418952597, 'gluc_weight': 0.9066618576673418, 'smoke_weight': 0.2535815475365083, 'alco_weight': 0.5500095290336634, 'active_weight': 0.6475556707330159}. Best is trial 0 with value: 0.7198266976990588.


...

Running Java anonymization for trial 48 with k=10...
Running Java code with k=10...
Solution space too large for k=10 and l=2: Attribute 'age': hierarchy does not contain a transformation rule for value '[0, 80['
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 2.63 Sekunden gedauert.
Anonymization completed for k=10. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_48_k10.csv.
Running Java anonymization for trial 48 with k=20...
Running Java code with k=20...
Solution space too large for k=20 and l=2: Attribute 'age': hierarchy does not contain a transformation rule for value '[0, 80['
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 2.78 Sekunden gedauert.
Anonymization completed for k=20. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_48_k20.csv.
All anonymization processes completed.


[I 2024-09-17 21:31:35,931] Trial 48 finished with value: 0.719198277738712 and parameters: {'age_weight': 0.5160793791126167, 'gender_weight': 0.5921864147035183, 'height_weight': 0.004486454482425621, 'weight_weight': 0.25954658934920416, 'ap_hi_weight': 0.14413113071626601, 'ap_lo_weight': 0.3817743706686768, 'cholesterol_weight': 0.5732705661979027, 'gluc_weight': 0.6734200553653723, 'smoke_weight': 0.1372667108805074, 'alco_weight': 0.8772552948506633, 'active_weight': 0.6273654209511015}. Best is trial 10 with value: 0.7268503515281477.


Running Java anonymization for trial 49 with k=10...
Running Java code with k=10...
Solution space too large for k=10 and l=2: Attribute 'age': hierarchy does not contain a transformation rule for value '[0, 80['
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 3.06 Sekunden gedauert.
Anonymization completed for k=10. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_49_k10.csv.
Running Java anonymization for trial 49 with k=20...
Running Java code with k=20...
Solution space too large for k=20 and l=2: Attribute 'age': hierarchy does not contain a transformation rule for value '[0, 80['
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 2.60 Sekunden gedauert.
Anonymization completed for k=20. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/anonymized_trial_49_k20.csv.
All anonymization processes completed.


[I 2024-09-17 21:31:53,770] Trial 49 finished with value: 0.7223818597876719 and parameters: {'age_weight': 0.3905656268086535, 'gender_weight': 0.45480369161343415, 'height_weight': 0.1354166704100852, 'weight_weight': 0.46755934536286115, 'ap_hi_weight': 0.006354628142436203, 'ap_lo_weight': 0.44382438870780194, 'cholesterol_weight': 0.787565202747427, 'gluc_weight': 0.5119548341995261, 'smoke_weight': 0.1556878966374508, 'alco_weight': 0.8356964413931307, 'active_weight': 0.5273026333035166}. Best is trial 10 with value: 0.7268503515281477.


Optimized weights for C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/cvd_k1000_l2.csv: {'age_weight': 0.4417194843109914, 'gender_weight': 0.9993626154243063, 'height_weight': 0.881040867694267, 'weight_weight': 0.058942964812388116, 'ap_hi_weight': 0.31156771428233454, 'ap_lo_weight': 0.5843905700190671, 'cholesterol_weight': 0.5814446569348688, 'gluc_weight': 0.3180476310226602, 'smoke_weight': 0.9514964164590098, 'alco_weight': 0.9714842983086067, 'active_weight': 0.3127374308669667}
Optimization results saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/optimized_results.csv.
